# Part 1: Getting some Data

I chose to look at crime data in the Los Angeles area which includes information about where each crime is commited, the type of the crime, the age of the victim, among others. The dataset can be found here: https://catalog.data.gov/dataset/crime-data-from-2020-to-present/resource/5eb6507e-fa82-4595-a604-023f8a326099?inner_span=True. I specifically used the csv file. 

In [1]:
import folium
import requests
import pandas

In [2]:
crime_table = pandas.read_csv("Crime_Data_from_2020_to_Present.csv")

crime_table

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617342,221213384,06/07/2022 12:00:00 AM,06/07/2022 12:00:00 AM,1900,12,77th Street,1249,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,...,AO,Adult Other,626.0,NaN,NaN,NaN,6600 S HOOVER ST,NaN,33.9792,-118.2871
617343,220314584,08/02/2022 12:00:00 AM,08/02/2022 12:00:00 AM,10,3,Southwest,334,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,4300 W 28TH ST,NaN,34.0311,-118.3335
617344,221005507,02/10/2022 12:00:00 AM,02/09/2022 12:00:00 AM,1530,10,West Valley,1024,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,18800 SHERMAN WY,NaN,34.2011,-118.5426
617345,221105477,02/10/2022 12:00:00 AM,02/08/2022 12:00:00 AM,2000,11,Northeast,1171,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,4000 FOUNTAIN AV,NaN,34.0958,-118.2787


There doesn't seem to be any missing latitude/longitudes:

In [3]:
crime_table[pandas.notnull(crime_table["LAT"])]["LAT"].count()

617347

In [4]:
crime_table[pandas.notnull(crime_table["LON"])]["LON"].count()

617347

In [5]:
# using a single column for coordinate rather than two separate latitude/longitude columns
crime_table["COR"] = list(zip(crime_table["LAT"], crime_table["LON"]))
crime_table.drop(columns=["LAT", "LON"], inplace = True)

# Part 2: Making a Map
The following code will create a map centered on Hollywood:

In [6]:
map_osm = folium.Map(location=[34.0928, -118.3287], zoom_start=13)
map_osm

# Part 3: Combining Parts 1 and 2

In [7]:
color_map = {'F':"red", 'M':"blue"}
crime_table[crime_table["Crm Cd"] == 812][crime_table["AREA"] == 6] \
.apply(lambda x: folium.Marker(list(x["COR"]), 
                               icon=folium.Icon(color=color_map[x["Vict Sex"]])
                              ).add_to(map_osm), 
       axis = 1)
map_osm

/tmp/ipykernel_31880/856623505.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  crime_table[crime_table["Crm Cd"] == 812][crime_table["AREA"] == 6] \


The map above is trying to show the distribution of crimes done in Hollywood (corresponds with value 6 in the "AREA" column in this dataset) against children, which includes all victims 13 or under as well as victims between 14-15 whose suspects are at least 10 years older (corresponds with value 812 in the "Crm Cd" column in this dataset). 

Specifically, it tries to visually compare the location of crimes against female children with the location of crimes against male children.

As seen above, the map reveals that most children victims are females, many of whom have been attacked along Hollywood Freeway, while attacks on male child victims are more spread out.